## Imports

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.callbacks import ReduceLROnPlateau

# Data Loader

So we can use big dataset

In [2]:
class DataSequence(tf.keras.utils.Sequence):
    
    def __init__(self, files, batch_size):
        self.files = files
        self.batch_size = batch_size
        file_lengths = [len(np.load(f)) for f in files]
        self.total_length = sum(file_lengths)
        self.cumulative_lengths = np.cumsum([0] + file_lengths)
        self.file_num = 0
        self.file_cache = pd.DataFrame(np.load(self.files[self.file_num])).sample(frac=1)

    def __len__(self):
        return int(np.ceil(self.total_length) / self.batch_size)
    
    def __getitem__(self, idx):
        # Start and end lengths of batch
        global_start = idx * self.batch_size
        global_end = global_start + self.batch_size

        # Get global bounds of current file
        fileLowerBound, fileUpperBound = self.cumulative_lengths[self.file_num], self.cumulative_lengths[self.file_num + 1]

        # check that the global_start and global_end are within the bounds of the current  file
        # If not, then enter this. Here, we fetch the next file.
        if global_end > fileUpperBound:
            self.file_num  += 1
            self.file_cache = pd.DataFrame()  # Clear the cache by setting it to an empty DataFrame
            self.file_cache = pd.DataFrame(np.load(self.files[self.file_num]), dtype=np.int16).sample(frac=1)

        # Enter this every epoch, or when batch number resets to 0        
        if global_start < fileLowerBound and global_start == 0:
            self.file_num = 0
            self.file_cache = pd.DataFrame()  # Clear the cache by setting it to an empty DataFrame
            self.file_cache = pd.DataFrame(np.load(self.files[self.file_num])).sample(frac=1)

        fileLowerBound, fileUpperBound = self.cumulative_lengths[self.file_num], self.cumulative_lengths[self.file_num + 1]

        
        local_start = global_start - fileLowerBound
        local_end = local_start + self.batch_size
        data = self.file_cache.iloc[local_start:local_end]

        x = data.iloc[:, :-1].to_numpy().astype(np.int8)
        y = data.iloc[:, -1].to_numpy().astype(np.int16)

        return (x, y)

### Fitting Model

Need to rerun the processedDataset, such that the evaluations are correct.

PARAMETERS BELOW

In [3]:
model = tf.keras.Sequential([
    # tf.keras.layers.Reshape((8, 8, 13), input_shape=(832,)),
    # tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal'),
    # tf.keras.layers.MaxPooling2D((2, 2)),
    # tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal'),
    # tf.keras.layers.MaxPooling2D((2, 2)),
    # tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dense(1024, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dense(1024, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1024, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dense(512, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dense(64, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dense(1, activation='linear', kernel_initializer='he_normal')
])

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, clipnorm=1.0)
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.01, clipnorm = 1.0)

model.compile(optimizer=optimizer, loss='MeanSquaredError', metrics=['MeanAbsoluteError'])

In [4]:
batch_size = 8192
files = ['./../data/npyV1/mixedDataChunk' + str(num) +'.npy' for num in np.arange(13)]

# Make the DataSequence object to pass data to model
train = DataSequence(files[0:5], batch_size = batch_size)

valid = DataSequence(files[5:6], batch_size = batch_size)

In [5]:
# validation checking
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Reduce learning rate
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.00001, verbose=1)

model.fit(train, epochs=60, validation_data = valid, callbacks=[early_stopping, reduce_lr], shuffle = False)

model.save("../saved_models/lastlastlastlastmodel")

Epoch 1/60
610/610 [==============================] - 25s 38ms/step - loss: 248119.0781 - mean_absolute_error: 263.9427 - val_loss: 220305.8594 - val_mean_absolute_error: 256.9581 - lr: 0.0100
Epoch 2/60
610/610 [==============================] - 24s 36ms/step - loss: 208637.4531 - mean_absolute_error: 251.5627 - val_loss: 193414.7812 - val_mean_absolute_error: 242.0929 - lr: 0.0100
Epoch 3/60
610/610 [==============================] - 23s 34ms/step - loss: 193946.5938 - mean_absolute_error: 244.6238 - val_loss: 185745.8125 - val_mean_absolute_error: 236.0078 - lr: 0.0100
Epoch 4/60
610/610 [==============================] - 24s 37ms/step - loss: 185976.0938 - mean_absolute_error: 239.3477 - val_loss: 193547.3125 - val_mean_absolute_error: 257.6861 - lr: 0.0100
Epoch 5/60
610/610 [==============================] - 23s 35ms/step - loss: 178096.1406 - mean_absolute_error: 233.4611 - val_loss: 182773.7500 - val_mean_absolute_error: 245.7703 - lr: 0.0100
Epoch 6/60
610/610 [===============